# RNN을 이용한 텍스트 생성
```
'경마장에 있는 말이 뛰고 있다'
'그의 말이 법이다'
'가는 말이 고와야 오는 말이 곱다'
```

In [1]:
import numpy as np
import tensorflow as tf
seed = 2022
np.random.seed(seed)
tf.random.set_seed(seed)

In [2]:
text = '''경마장에 있는 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다'''

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [4]:
# 단어 집합 생성
t = Tokenizer()
t.fit_on_texts([text])

In [5]:
# 특징
# 1. word_index 순서가 가나다 순서가 아니고, 많이 나온 단오 순이다.
# 2. 0 index가 없다
t.word_index

{'가는': 8,
 '경마장에': 2,
 '고와야': 9,
 '곱다': 11,
 '그의': 6,
 '뛰고': 4,
 '말이': 1,
 '법이다': 7,
 '오는': 10,
 '있는': 3,
 '있다': 5}

In [6]:
# 단어 집합 크기 - Keras Tokenizer의 시작 인덱스가 1이기 때문에
vocab_size = len(t.word_index) + 1  # 마지막 인덱스 값을 포함하기 위해서
vocab_size

12

In [7]:
t.texts_to_sequences(['그의 말이 법이다'])

[[6, 1, 7]]

In [8]:
for line in text.split('\n'):
    encoded = t.texts_to_sequences([line])[0]
    print(encoded)

[2, 3, 1, 4, 5]
[6, 1, 7]
[8, 1, 9, 10, 1, 11]


In [9]:
sequences = []
for line in text.split('\n'):
    encoded = t.texts_to_sequences([line])[0]
    for i in range(1,len(encoded)):
        sequences.append(encoded[:i+1])

sequences

[[2, 3],
 [2, 3, 1],
 [2, 3, 1, 4],
 [2, 3, 1, 4, 5],
 [6, 1],
 [6, 1, 7],
 [8, 1],
 [8, 1, 9],
 [8, 1, 9, 10],
 [8, 1, 9, 10, 1],
 [8, 1, 9, 10, 1, 11]]

In [10]:
max_len = max(len(seq) for seq in sequences)
max_len

6

- Padding
    - 전체 샘플 길이를 max_len에 맞추도록 0를 추가
    - pre 옵션을 주면 앞쪽을 채움

In [11]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
sequences[:5]

array([[0, 0, 0, 0, 2, 3],
       [0, 0, 0, 2, 3, 1],
       [0, 0, 2, 3, 1, 4],
       [0, 2, 3, 1, 4, 5],
       [0, 0, 0, 0, 6, 1]], dtype=int32)

-> 앞 5개 숫자 X, 마지막 숫자 y

In [12]:
X = sequences[:, :-1]
y = sequences[:, -1]

In [13]:
X.shape

(11, 5)

In [14]:
Y = to_categorical(y)
Y[:3]

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

### 모델 정의
- Embedding
- SimpleRNN
- Dense

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

In [16]:
# Embedding=4, SimpleRNN=32
model = Sequential([ 
    Embedding(vocab_size, 4, input_length=max_len-1),
    SimpleRNN(32),
    Dense(vocab_size, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 5, 4)              48        
                                                                 
 simple_rnn (SimpleRNN)      (None, 32)                1184      
                                                                 
 dense (Dense)               (None, 12)                396       
                                                                 
Total params: 1,628
Trainable params: 1,628
Non-trainable params: 0
_________________________________________________________________


### 모델 설정/학습

In [17]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
hist = model.fit(X, Y, epochs=200, verbose=0)

In [19]:
# 정확도 100%
hist.history['accuracy'][-1]

1.0

### 모델 검증

In [20]:
from google.colab import files
up = files.upload()

Saving my_util.py to my_util.py


In [21]:
from my_util import sentence_generation

In [22]:
# 모델, 토크나이저, maxlen, 현재 단어, 반복할 횟수
sentence_generation(model, t, max_len, '경마장에', 3)

'경마장에 있는 말이 뛰고'

In [23]:
sentence_generation(model, t, max_len, '가는', 5)

'가는 말이 고와야 오는 말이 곱다'

### 모델 변화
- Embedding vector 갯수: 2,4,6
- RNN 유닛 갯수: 24, 32, 48

In [24]:
for n_embed in [2,4,6]:
    for n_unit in [24,32,48]:
        model = Sequential([ 
            Embedding(vocab_size, n_embed, input_length=max_len-1),
            SimpleRNN(n_unit),
            Dense(vocab_size, activation='softmax')
        ])
        model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])
        hist = model.fit(X, Y, epochs=200, verbose=0)
        print('====================================')
        print(f'Embedding vector: {n_embed}, RNN unit: {n_unit}')
        print(f"정확도: {hist.history['accuracy'][-1]:.4f}")
        print(sentence_generation(model, t, max_len, '경마장에', 4))
        print(sentence_generation(model, t, max_len, '그의', 2))
        print(sentence_generation(model, t, max_len, '가는', 5))
        print()

Embedding vector: 2, RNN unit: 24
정확도: 0.8182
경마장에 말이 말이 오는 말이
그의 말이 법이다
가는 말이 법이다 오는 말이 곱다

Embedding vector: 2, RNN unit: 32
정확도: 0.8182
경마장에 말이 법이다 오는 말이
그의 말이 고와야
가는 말이 고와야 오는 말이 곱다

Embedding vector: 2, RNN unit: 48
정확도: 0.9091
경마장에 말이 고와야 오는 말이
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

Embedding vector: 4, RNN unit: 24
정확도: 0.8182
경마장에 말이 말이 오는 말이
그의 말이 법이다
가는 말이 법이다 오는 말이 곱다

Embedding vector: 4, RNN unit: 32
정확도: 1.0000
경마장에 있는 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

Embedding vector: 4, RNN unit: 48
정확도: 0.9091
경마장에 말이 고와야 오는 말이
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

Embedding vector: 6, RNN unit: 24
정확도: 1.0000
경마장에 있는 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

Embedding vector: 6, RNN unit: 32
정확도: 1.0000
경마장에 있는 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

Embedding vector: 6, RNN unit: 48
정확도: 1.0000
경마장에 있는 말이 뛰고 있다
그의 말이 법이다
가는 말이 고와야 오는 말이 곱다

